In [166]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%run implementations.ipynb
%reload_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [143]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## First Model using least squares

In [144]:
# first to inspect the data, I notice many invalid values of -999
# for this first model I disregard them

In [146]:
# first to split the data into training and testing

In [147]:
ratio = 0.8
seed = 3

x_tr, x_te, y_tr, y_te = split_data(tX,y, ratio, seed)

In [148]:
# then to standardize x_tr and x_te

In [149]:
x_tr = standardize(x_tr)[0]
x_te = standardize(x_te)[0]

In [150]:
# next to add a column of ones

In [151]:
y_tr, x_tr = build_model_data(x_tr, y_tr)
y_te, x_te = build_model_data(x_te, y_te)

In [152]:
# First Model using least squares

weights = least_squares(y_tr, x_tr)
MSE_tr = compute_loss(y_tr, x_tr, weights)
MSE_te = compute_loss(y_te, x_te, weights)

print(MSE_tr, MSE_te) #0.33927307620272856 11.175145651957722

0.33927307620272856 11.175145651957722


## Second Model with pre-processing of N/A data

In [159]:
# removing all nan but there actually are none
tX = tX[~np.isnan(tX).any(axis=1)]

In [170]:
# removing all rows with -999, there are 181886
y = y[np.all(tX != -999, axis=1)]
tX = tX[np.all(tX != -999, axis=1)]

x_tr, x_te, y_tr, y_te = split_data(tX,y, ratio, seed)

x_tr = standardize(x_tr)[0]
x_te = standardize(x_te)[0]

y_tr, x_tr = build_model_data(x_tr, y_tr)
y_te, x_te = build_model_data(x_te, y_te)

weights = least_squares(y_tr, x_tr)
MSE_tr = compute_loss(y_tr, x_tr, weights)
MSE_te = compute_loss(y_te, x_te, weights)

print(MSE_tr, MSE_te) #0.36864162046559024 0.5411866496785176

0.36864162046559024 0.5411866496785176


## Third Model with Feature Augmentation

In [ ]:

degrees = [1, 2, 3, 4]

# define the structure of the figure
num_row = 2
num_col = 2
f, axs = plt.subplots(num_row, num_col)
    
for ind, degree in enumerate(degrees):

        polynome = build_poly(x_tr,degree)
        w = least_squares(y_tr, polynome.T)
        MSE = compute_loss(y_tr, polynome, w)

        print("Processing {i}th experiment, degree={d}, mse={loss}".format( 
            i=ind + 1, d=degree, loss=MSE))
#         plot_fitted_curve(y_tr, x_tr, weights, degree, axs[ind // num_col][ind % num_col])
#         plt.tight_layout()
#         plt.show()


## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)